<h1>Mission Phase Resolved Movie</h1>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy import signal
from scipy import interpolate
from scipy import optimize
from scipy import io
import cftime
import netCDF4 as nc
import pickle
import glob
import scipy.stats as st
from PyAstronomy import pyaC
import cmocean
import cftime
from scipy.stats import gaussian_kde
import scipy.stats
import matplotlib.animation as animation
%matplotlib widget

In [2]:
# Import Data from Mission 15
mission_dataset = nc.Dataset('../microSWIFT_data/cleanedDataset/mission_15.nc')

# Scatter the Cross Shore Location with the sea surface elevation for each time step for each microSWIFT
microSWIFTs_on_mission = list(mission_dataset.groups.keys())

# Plot the sea surface elevation as a function of cross shore location 
fig = plt.figure()
ax1 = plt.subplot(2,1,1)
ax2 = plt.subplot(2,1,2)

  # Add the FRF Bathymetry to the map 
# Data from September 28th, 2021
# bathy_url = 'https://chlthredds.erdc.dren.mil/thredds/dodsC/frf/geomorphology/DEMs/surveyDEM/data/FRF_geomorphology_DEMs_surveyDEM_20210928.nc'
bathy_file = '../microSWIFT_data/FRFdata/FRF_geomorphology_DEMs_surveyDEM_20211021.nc'
bathy_dataset = nc.Dataset(bathy_file)
# Create grid from coordinates
xFRF_grid, yFRF_grid = np.meshgrid(bathy_dataset['xFRF'][:],bathy_dataset['yFRF'][:])
bathy = bathy_dataset['elevation'][0,:,:]

# Fixed Instrument Locations
awac4p5m_location = [397.35, 890.98] # Converted from lat lon locations published on FRF data portal
awac6m_location = [603.12, 886.68]   # Converted from lat lon locations published on FRF data portal
array8m_location = [914.05, 826.08]  # Nominal location based on published data even though this array has multiple locations

def init_scatter():
        ax1.clear()
        ax1.set_xlabel('FRF X Location [meters]')
        ax1.set_ylabel('Sea Surface Elevation[meters]')
        ax2.set_xlabel('FRF X Location [meters]')
        ax2.set_ylabel('FRF Y Location [meters]')
        im = ax2.contourf(xFRF_grid, yFRF_grid, bathy, cmap=cmocean.cm.deep_r)
        for microSWIFT in microSWIFTs_on_mission:
                eta = mission_dataset[microSWIFT]['eta'][6000] 
                y = mission_dataset[microSWIFT]['yFRF'][6000]
                x = mission_dataset[microSWIFT]['xFRF'][6000]
                ax1.scatter(x, eta, color='r')
                ax2.scatter(x, y, color='r')

def update_scatter(index):
        ax1.clear()
        ax2.clear()
        ax1.set_xlabel('FRF X Location [meters]')
        ax1.set_ylabel('Sea Surface Elevation[meters]')
        ax2.set_xlabel('FRF X Location [meters]')
        ax2.set_ylabel('FRF Y Location [meters]')
       
        # Get list of all microSWIFTs on the mission
        x_loc = []
        eta_loc = []
        im = ax2.contourf(xFRF_grid, yFRF_grid, bathy, cmap=cmocean.cm.deep_r)
        for microSWIFT in microSWIFTs_on_mission:
                ax1.plot(mission_dataset[microSWIFT]['xFRF'][index], mission_dataset[microSWIFT]['eta'][index], color='r', marker='o')
                ax2.plot(mission_dataset[microSWIFT]['xFRF'][index], mission_dataset[microSWIFT]['yFRF'][index], color='r', marker='o')
                ax1.set_xlim(200, 350)
                ax1.set_ylim(-4, 4)
                ax2.set_xlim(200, 350)
                ax2.set_ylim(0, 400)
                x_loc.append(mission_dataset[microSWIFT]['xFRF'][index])
                eta_loc.append(mission_dataset[microSWIFT]['eta'][index])

        # Plot the surface elevation from the buoys
        x_loc = np.array(x_loc)
        eta_loc = np.array(eta_loc)
        inds = np.argsort(x_loc)
        x_loc_sort = x_loc[inds]
        eta_loc_sort = eta_loc[inds]
        x_array = np.linspace(np.min(x_loc), np.max(x_loc))
        eta_interp_func = interpolate.interp1d(x_loc, eta_loc)
        num = 500
        x_array = np.linspace(np.nanmin(x_loc), np.nanmax(x_loc), 500)
        eta_interped = eta_interp_func(x_array)
        ax1.plot(x_array, eta_interped)

        # Add title with time
        time = cftime.num2pydate(mission_dataset['time'][index], units=mission_dataset['time'].units, calendar=mission_dataset['time'].calendar)
        ax1.set_title('Time - {}'.format(time))
        fig.tight_layout()

# Create Animation object
data_skip = 1
time = mission_dataset['time'][:]
print(time.size)
anim = animation.FuncAnimation(fig, update_scatter, frames=np.arange(11000, 12000, data_skip), init_func=init_scatter)
anim.save('./MissionPhaseResolved.gif', dpi=150, fps=12, writer='pillow')

# Close dataset and figures
mission_dataset.close()
plt.close()

15121
